<a href="https://colab.research.google.com/github/hsnbnaa/Tugas_BigData/blob/main/Kelompok13_CodeScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install lzma pandas requests tqdm

ERROR: Could not find a version that satisfies the requirement lzma (from versions: none)
ERROR: No matching distribution found for lzma


In [ ]:
import requests, lzma, struct
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta

def download_bi5(symbol, year, month, day, hour):
    url = f"https://datafeed.dukascopy.com/datafeed/{symbol}/{year:04d}/{month-1:02d}/{day:02d}/{hour:02d}h_ticks.bi5"
    try:
        r = requests.get(url, timeout=10)
        if r.status_code != 200:
            return None
        decompressed = lzma.decompress(r.content)
    except:
        return None

    data = []
    for i in range(0, len(decompressed), 20):
        chunk = decompressed[i:i+20]
        if len(chunk) < 20:
            break
        msc, ask, bid, ask_vol, bid_vol = struct.unpack('>LLLff', chunk)
        data.append(bid / 1e5)  # pakai BID sebagai harga utama
    return data if data else None


def get_hourly_ohlc(symbol, start_year=2024, end_year=2025):
    current = datetime(start_year, 1, 1)
    end = datetime(end_year, 12, 31, 23)
    result = []

    for _ in tqdm(range(int((end - current).total_seconds() // 3600) + 1), desc="Downloading"):
        data = download_bi5(symbol, current.year, current.month, current.day, current.hour)
        if data:
            open_ = data[0]
            high_ = max(data)
            low_ = min(data)
            close_ = data[-1]
            volume_ = len(data)
            result.append([current.strftime("%Y-%m-%d %H:%M:%S"), open_, high_, low_, close_, volume_])
        current += timedelta(hours=1)

    df = pd.DataFrame(result, columns=["date", "open", "high", "low", "close", "volume"])
    return df

symbol = "XAUUSD"
df = get_hourly_ohlc(symbol, start_year=2025, end_year=2025)
df.to_csv(f"{symbol.lower()}_2000_2025_1h.csv", index=False)
print("✅ Data berhasil disimpan:", f"{symbol.lower()}_2000_2025_1h.csv")

Downloading:  69%|██████▉   | 6048/8760 [1:21:01<21:12,  2.13it/s]